# Import

### Basic

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import time

In [2]:
%matplotlib inline

In [3]:
import params

# Load dataset

In [4]:
DATASET = np.load("data/dataset.npy")

In [5]:
print "Shape :", DATASET.shape

Shape : (5520598L, 160L, 1L)


### Splitting into a trainset, a validset and a testset

In [6]:
def split_dataset(dataset, split_rate=[0.8, 0.1, 0.1], seed=123): # see preprocessing.py
    # index = 1,2,3,...,N
    N = dataset.shape[0]
    index = range(dataset.shape[0])
    # shuffle
    np.random.seed(seed)
    np.random.shuffle(index)
    # division
    train_rate, valid_rate, test_rate = np.cumsum(split_rate)
    index_train = index[0:int(train_rate*N)]
    index_valid = index[int(train_rate*N):int(valid_rate*N)]
    index_test = index[int(valid_rate*N):int(test_rate*N)]
    # return 
    return index_train, index_valid, index_test

In [7]:
index_train, index_valid, index_test = split_dataset(DATASET)

**index_train, index_valid, index_test** are such that :

trainset = DATASET[index_train]

validset = DATASET[index_valid]

testset = DATASET[index_test]

# Preprocessing function

In [8]:
ACCEPTED_CHARS = params.ACCEPTED_CHARS

In [9]:
print ACCEPTED_CHARS

['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x80', '\x92', '\x98', '\x9f', '\xa6', '\xe2', '\xf0']


In [10]:
def batch2onehot(batch, D): # see preprocessing.py
    ''' Function used during the training to encode batches.
    Input size : (batch_size, tweet_length, 1).
    Output size : (batch_size, tweet_length, D)'''
    B, T = batch.shape[0:2]
    one_hot_batch = np.zeros((B*T, D))
    one_hot_batch[range(B*T), batch.flatten()] = 1
    one_hot_batch = one_hot_batch.reshape((B,T,D))
    return one_hot_batch

def batch2tweet(batch, accepted_caracters, special_char=""): # see preprocessing.py
    '''Not optimized. But not used during the training : no need to be fast.'''
    tweets = []
    for t in batch:
        tweet = ""
        for char in t:
            try:
                tweet += accepted_caracters[char[0]]
            except:
                tweet += special_char # Special marker indicating the end of the tweet
        tweets.append(tweet)
    return tweets

def onehot2tweet(batch, accepted_caracters, special_char=""): # see preprocessing.py
    '''Not optimized. But not used during the training : no need to be fast.'''
    tweets = []
    for t in batch:
        tweet = ""
        for char in t:
            try:
                tweet += accepted_caracters[np.where(char==1)[0][0]]
            except:
                tweet += special_char # Special marker indicating the end of the tweet
        tweets.append(tweet)
    return tweets

### Test preprocessing functions

In [84]:
D = params.D  # dimension of one-hot vectors
B = params.B  # batch size for the demo
T = params.T  # max length of a tweet
batch = DATASET[0:B] 
# Dataset > One-hot
one_hot_batch = batch2onehot(batch, D)

# Pick a tweet
t = np.random.randint(B)
# Pick a char
c = np.random.randint(T)
print "Char %d from tweet %d :"%(c,t), one_hot_batch[t][c]

Char 76 from tweet 40 : [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]


#### If you want to, possible to go back in the "string" domain

In [12]:
# Using 'batch2tweet' :
t = np.random.randint(B) # Pick a tweet
print batch2tweet(batch, ACCEPTED_CHARS)[t]
# or 'onehot2tweet' if the tweet has already been converted to onehot :
print onehot2tweet(one_hot_batch, ACCEPTED_CHARS)[t]

rt @kendrakuuddy: i pray to god every day not to ask him for more but to thank him for everything he already has blessed me with ��
rt @kendrakuuddy: i pray to god every day not to ask him for more but to thank him for everything he already has blessed me with ��


# Generator

In [13]:
def batch_generator(data, index, batch_size, D): # see preprocessing.py
    # Init iterator and shuffling the dataset
    count = 0
    np.random.shuffle(index)
    while 1:
        if count+batch_size >= len(index):
            # Reset counter and shuffling
            count = 0
            np.random.shuffle(index)
        # Get raw data
        raw_batch = np.copy(data[index[count:(count+batch_size)]])
        # One-hot encoding
        one_hot_batch = batch2onehot(raw_batch, D)
        # Remove the padding dimension
        one_hot_batch = one_hot_batch[:,:,0:(D-1)] # s.t. padding features are full of 0s
                                                   # and will be masked by the Masking layer
                                                   # (see below in the model definition)
        # Target
        input_batch  = one_hot_batch[:, 0:-1, :]
        target_batch = one_hot_batch[:, 1:, :] # target = 1-shifted input batch
        del raw_batch
        count += batch_size
        # Yield
        yield input_batch, target_batch

# Custom functions used to save stuff...

**save_architecture** : save the architecture of the model in a text file.

**create_log** : create a text file where the loss, and other metrics will be printed using **write_log**

**ModelSaver** : Custom Keras callback (https://keras.io/callbacks/). This object is given to the 'fit' (or equivalently 'fit_generator') when launching a training. It permits to control when to save weights, and underwhich name.

In [14]:
def save_architecture(model, path_out): # see logging.py
    """
    Based on the keras utils 'model.summary()'
    """
    # Redirect the print output the a textfile
    orig_stdout = sys.stdout
    # and store the architecture
    f = file(os.path.join(path_out, "architecture.txt"), 'w')
    sys.stdout = f
    model.summary()
    # Reset the print output direction
    sys.stdout = orig_stdout
    f.close()

    open(os.path.join(path_out, "config.json"), 'w').write(model.to_json())


def create_log(path, settings, filename="log.txt"): # see logging.py
    f = open(os.path.join(path, filename), "w")
    f.writelines(str(settings))
    f.writelines("\n####\nStarted on %s at %s\n" % (time.strftime("%d/%m/%Y"), time.strftime("%H:%M:%S")))
    f.close()


def write_log(path, string, filename="log.txt"): # see logging.py
    """
    Add a line at the end of a textfile.

    :param path: textfile location
    :param string: line to add
    """
    # Open and Read
    f = open(os.path.join(path, filename), "r")
    lines = f.readlines()
    f.close()
    # Adding a line
    lines.append(string)
    # Write
    f = open(os.path.join(path, filename), "w")
    f.writelines(lines)
    f.close()


from keras.callbacks import Callback
    
class ModelSaver(Callback): # see logging.py
    """
    Keras callback subclass which defines a saving procedure of the model being trained : after each epoch,
    the last model is saved under the name 'after_random.cnn'. The best model is saved with the name 'best_model.cnn'.
    The model after random can also be saved. And the model architecture is saved with the name 'config.network'.
    Everything is stored using pickle.
    """

    def __init__(self, path, path_weights, monitor, verbose=1):
        super(Callback, self).__init__()
        self.verbose = verbose
        self.path = path
        self.path_weights = path_weights
        self.monitor = monitor
        self.best = np.Inf
        

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.epoch_end = time.time()
        # get loss
        monitor = logs.get(self.monitor)
        # condition = True if loss decreased
        condition = monitor < self.best

        if condition:
            # Save weights as "best_model.weights"
            self.best = monitor
            save_path = os.path.join(self.path_weights, "best_model.weights")
            self.model.save_weights(save_path, overwrite=True)
        else:
            # Save weights as "last_epoch.weights"
            save_path = os.path.join(self.path_weights, "last_epoch.weights")
            self.model.save_weights(save_path, overwrite=True)
        
        # Log file management
        if self.verbose > 0:
            log_string = "####\nEpoch %d took %d s: " % (epoch, int(self.epoch_end-self.epoch_start))
            for k in logs.keys():
                log_string += "%s : %.4f # " % (k, logs.get(k))
            if condition:
                log_string += "\tBEST"
            write_log(self.path, log_string)

Using Theano backend.
Using gpu device 0: GeForce GTX 850M (CNMeM is disabled, cuDNN 5005)


# Model definition

In [15]:
from keras.models import Model
from keras.layers import Input, LSTM, Masking, Dropout, TimeDistributed, Dense, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [16]:
def get_model(T, D, lr, nhidden, drop_rate): # see models.py
    # Input layer
    inputs = Input((T, D))
    # Masking "only-0" input features
    masked = Masking(mask_value=0.0)(inputs)
    # Hidden layers
    for i in range(nhidden):
        if i == 0:
            hidden  = LSTM(128, return_sequences=True)(masked)
        else:
            hidden  = LSTM(128, return_sequences=True)(dropout)
        dropout = Dropout(drop_rate)(hidden)
    # Output layer : linear TimeDistributedDense + softmax
    decoder = TimeDistributed(Dense(D))(dropout) # Apply the same dense layer on each timestep
    outputs = Activation("softmax") (decoder)

    model = Model(input=inputs, output=outputs)

    model.compile(optimizer=Adam(lr=lr), loss="categorical_crossentropy")

    return model

### Model instanciation

In [17]:
LR = params.LR # learning rate
model = get_model(T-1, D-1, LR, 3, 0.1) # D-1 because params.D accounts for the padding dimension

In [18]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 159, 63)       0                                            
____________________________________________________________________________________________________
masking_1 (Masking)              (None, 159, 63)       0           input_1[0][0]                    
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 159, 128)      98304       masking_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 159, 128)      0           lstm_1[0][0]                     
___________________________________________________________________________________________

### Some test on random inputs

In [70]:
input_sequence = np.random.randn(T*(D-1)).reshape((1, T, D-1))

In [71]:
model.predict(input_sequence).shape

(1L, 160L, 63L)

# Training

In [37]:
PATH_EXPERIMENT = params.PATH_EXPERIMENT
NB_EPOCHS = params.NB_EPOCHS
NB_SAMPLES_PER_EPOCH = params.NB_SAMPLES_PER_EPOCH
PATIENCE = params.PATIENCE

### Training function

In [77]:
def trainargs2strings(path, model, dataset, index_train, index_valid, D, batch_size, 
             nsamples_per_epoch, nepoch, patience): # see logging.py
    settings = ""
    settings += "Path : %s"%path
    settings += "\nDataset shape :" + str(dataset.shape)
    settings += "\nNtrain : %d"%len(index_train)
    settings += "\nNvalid : %d"%len(index_valid)
    settings += "\nBatch size : %d"%batch_size
    settings += "\nNb samples per epoch : %d"%nsamples_per_epoch
    settings += "\nNb epochs : %d"%nepoch
    settings += "\nPatience : %d"%patience
    return settings

In [117]:
def training(path, model, dataset, index_train, index_valid, D, batch_size, 
             nsamples_per_epoch, nepoch, patience): # see training.py
    start = time.time()
    # Create dir (if not already done)
    if os.path.exists(path) is False:
        os.mkdir(os.path.abspath(path))
    path_weights = os.path.join(path, "weights")
    if os.path.exists(path_weights) is False:
        os.mkdir(os.path.abspath(path_weights))
    # Create log file
    settings = trainargs2strings(path, model, dataset, index_train, index_valid, D, batch_size, 
                                 nsamples_per_epoch, nepoch, patience)
    create_log(path, settings)
    # Save architecture
    save_architecture(model, path)
    # Save weights after initialization
    model.save_weights(os.path.join(path_weights, "after_initialization.weights"), 
                       overwrite=True)

    # Callbacks
    early_stopping = EarlyStopping(monitor="val_loss", patience=patience)
    model_saver = ModelSaver(path, os.path.join(path, "weights"), monitor="val_loss")
    # Argument to give to generators
    train_generator_args = [dataset, index_train, batch_size, D]
    valid_generator_args = [dataset, index_valid, batch_size, D]
    # Training loop
    h = model.fit_generator(batch_generator(*train_generator_args), nsamples_per_epoch, nepoch, 
                            validation_data=batch_generator(*valid_generator_args), 
                            nb_val_samples=len(index_valid),
                            callbacks=[early_stopping, model_saver])

    end = time.time()
    print "Training took %.2fs"%(end-start)

### Training test

In [85]:
# Quick test
b = 256 # batch size
n = b*50 # number of examples in 1 epoch
nepoch = 3 # number of epochs
patience = 3 # patience

training(PATH_EXPERIMENT, model, DATASET, index_train, index_valid[0:(b*50)], D, 
         b, n, nepoch, patience)

NameError: name 'training' is not defined

# Test

Needs to be filled - Sampling functions - "Temperature"

In [88]:
model.load_weights("best_model.weights")

In [ ]:
first_character = "#"
i = np.where(np.array(ACCEPTED_CHARS)==first_character)[0]

sequence = np.zeros((1, T-1, D-1), "float32")
sequence[0,0,i] = 1

print first_character,
for t in range(T-2):
    out = model.predict(sequence)   
    next_char = np.random.choice(range(D-1), p=out[0,t]/out[0,t].sum())
    sequence[0,t+1,next_char] = 1
    if next_char != 62:
        char = ACCEPTED_CHARS[next_char]
    else:
        break
    print char,
    
onehot2tweet(sequence, ACCEPTED_CHARS)

# n o w p l a y i n g 

In [75]:
sequence

array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]], dtype=float32)